# Intro to Pandas 2, Polars, and DuckDB

© 2023 Matt Harrison

## About Matt  Harrison @\_\_mharrison\_\_

* Corporate trainer at MetaSnake. Taught Pandas to 1000's of students
* Author of Effective Pandas, Machine Learning Pocket Reference, and Illustrated Guide to Python 3.
* Advisor at Ponder (creators of Modin, sold to Snowflake)


## Relevant Background

* 1999 NLP
* 2006 Created Python OLAP Engine
* 2009 Heard about Pandas
* Used Pandas for failure modeling, analytics, and ml
* 2016 Learning the Pandas Library
* 2019 Spark
* 2020 Pandas Cookbook
* 2021 Effective Pandas
* 2022 CuDf, Modin, Polars

## Pandas 2.0

What's new in Pandas 2?

Two big things:

- Pyarrow types
- Copy on write

In [ ]:
!pip install -U polars duckdb jupysql pyarrow pandas

### Sample Problem

In [ ]:
import pandas as pd
import io
billing_data = \
'''cancel_date,period_start,start_date,end_date,rev,sum_payments
12/1/2019,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,1950
1/20/2020,1/1/2020,12/15/2019,5/15/2020,499,0
,1/1/2020,12/24/2019,5/24/2020,699,100
,1/1/2020,11/29/2019,4/29/2020,799,250
,1/1/2020,1/15/2020,4/29/2020,799,250'''

bill_df = pd.read_csv(io.StringIO(billing_data),
    dtype_backend='pyarrow',                 
    parse_dates=['cancel_date', 'period_start', 'start_date',
                 'end_date'])


In [ ]:
bill_df

In [ ]:
bill_df.dtypes

In [ ]:
def tweak_bill202(df_):
    return (df_
            .assign(cancel_date=pd.to_datetime(
                df_.cancel_date.replace('<NA>', ''), format='%m/%d/%Y')
                   )
           )

tweak_bill202(bill_df).dtypes

In [ ]:
(tweak_bill202(bill_df)
.astype({'cancel_date': 'timestamp[ns][pyarrow]',})
.astype({col: 'timestamp[ns][pyarrow]' for col in
         ['period_start', 'start_date', 'end_date']})
.dtypes
)

In [ ]:
def tweak_bill203(df_):
    return (df_
            .assign(cancel_date=pd.to_datetime(
                df_.cancel_date.replace('<NA>', ''), format='%m/%d/%Y')
                   )
            .astype({'cancel_date': 'timestamp[ns][pyarrow]',})
            .astype({col: 'timestamp[ns][pyarrow]' for col in
                        ['period_start', 'start_date', 'end_date']})
              )

tweak_bill203(bill_df).dtypes
            

### Apply (mis)solution


Suppose we are dealing with a company that offers a monthly subscription service. Customers can cancel their subscription at any time, but they are billed at the beginning of each month. In this scenario, we need to calculate the unbilled receivables at the end of each month. Unbilled receivables are the revenues that the company has earned but has not yet billed to the customer.


In [ ]:
import numpy as np
def calc_unbilled_rec(vals):
    cancel_date, period_start, start_date, end_date, rev, \
         sum_payments = vals
    if not pd.isna(cancel_date) and (cancel_date < period_start):
        return np.nan
    if start_date < period_start and end_date > period_start:
        if rev > sum_payments:
            return rev - sum_payments
        else:
            return 0

(tweak_bill203(bill_df)
.assign(unbilled_rec=lambda df_: df_.apply(calc_unbilled_rec, axis=1))
)

In [ ]:
def calc_unbilled_rec_vectorized(df):
    # Conditions
    condition_cancel_date = df['cancel_date'].isna() | (df['cancel_date'] >= df['period_start'])
    condition_active_subscription = (df['start_date'] < df['period_start']) & (df['end_date'] > df['period_start'])
    condition_revenue_greater = df['rev'] > df['sum_payments']

    # Apply conditions using pd.DataFrame.where
    return ((df['rev'] - df['sum_payments'])
            .where(condition_revenue_greater & condition_active_subscription & condition_cancel_date, 0)
            .where(condition_active_subscription & condition_cancel_date, np.nan)
    )



(
    tweak_bill203(bill_df)
    .assign(unbilled_rec=calc_unbilled_rec_vectorized)
)

In [ ]:
bill_100k = tweak_bill203(bill_df.sample(100_000, replace=True))

In [ ]:
%%timeit
(bill_100k
.assign(unbilled_rec=lambda df_: df_.apply(calc_unbilled_rec, axis=1))
)

In [ ]:
%%timeit
(bill_100k
    .assign(unbilled_rec=calc_unbilled_rec_vectorized)
#.assign(unbilled_rec=lambda df_: df_.apply(calc_unbilled_rec, axis=1))
)

### Numba Version for Fun

In [ ]:
import numpy as np
from numba import jit

@jit
def calc_unbilled_numba(cancel_date, period_start, start_date, 
                        end_date, rev, sum_payments):
    results = np.full(rev.shape[0], np.nan, dtype=np.float64)
    for i in range(rev.shape[0]):
        cd = cancel_date[i]
        ps = period_start[i]
        sd = start_date[i]
        ed = end_date[i]
        if cd > 0 and cd < ps:
            results[i] = np.nan
        elif sd < ps < ed:
            if rev[i] > sum_payments[i]:
                results[i] = rev[i] - sum_payments[i]
            else:
                results[i] = 0
    return results


In [ ]:
%%timeit
(bill_100k
    .assign(unbilled_rec=calc_unbilled_numba(*(ser.astype(int).to_numpy() 
                          for name, ser in bill_100k.items())))
)

### Questions about Pandas 2?


## Polars

* Leverages Arrow (Rust)
* Polars is a Rust library with Python bindings
* Polars has expressions and contexts
  - Contexts - `.select`, `.with_columns`, `.filter`
  - Expressions - done w/ `pl.col("col_name")` or `pl.lit(1)`
* Lazy evaluation
* Query optimization
* Multi-threaded

In [ ]:
import polars as pl
import io
billing_data = \
'''cancel_date,period_start,start_date,end_date,rev,sum_payments
12/1/2019,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,1950
1/20/2020,1/1/2020,12/15/2019,5/15/2020,499,0
,1/1/2020,12/24/2019,5/24/2020,699,100
,1/1/2020,11/29/2019,4/29/2020,799,250
,1/1/2020,1/15/2020,4/29/2020,799,250'''

bill_pl = pl.read_csv(io.StringIO(billing_data))
                      
bill_pl

In [ ]:
bill_pl.dtypes

In [ ]:
(bill_pl
 .with_columns(pl.col(['cancel_date', 'period_start', 'start_date','end_date'])
     .str.strptime(pl.Date, '%m/%d/%Y'))
 )

In [ ]:
# convert this to polars expression
import numpy as np
def calc_unbilled_rec(vals):
    cancel_date, period_start, start_date, end_date, rev, \
         sum_payments = vals
    if not pd.isna(cancel_date) and (cancel_date < period_start):
        return np.nan
    if start_date < period_start and end_date > period_start:
        if rev > sum_payments:
            return rev - sum_payments
        else:
            return 0

def calc_unbilled_pl():
    cancel = pl.col('cancel_date')
    period = pl.col('period_start')
    start = pl.col('start_date')
    end = pl.col('end_date')
    rev = pl.col('rev')
    sum_payments = pl.col('sum_payments')
    res = (pl.when(~cancel.is_null() & (cancel < pl.col('period_start')))
              .then(None)
             .when((start < period) & (end > period) & (rev > sum_payments))
              .then(rev - sum_payments)
             .when((start < period) & (end > period))
              .then(0)
             .otherwise(None)
    )
    return res

(bill_pl
 .with_columns(pl.col(['cancel_date', 'period_start', 'start_date','end_date'])
     .str.strptime(pl.Date, '%m/%d/%Y'))
    .with_columns(unbilled_rec=calc_unbilled_pl())
 )
        

In [ ]:
pl_100k = pl.from_pandas(bill_100k)

In [ ]:
%%timeit
(pl_100k
.with_columns(unbilled_rec=calc_unbilled_pl())
)

### Lazy and Query Optimization

In [ ]:
pl_100k.write_csv('/tmp/bill_100k.csv', datetime_format='%m/%d/%Y')

In [ ]:
bill_pl_lazy = pl.scan_csv('/tmp/bill_100k.csv')
def tweak_pl(df_):
    return (df_
        .with_columns(pl.col(['cancel_date', 'period_start', 'start_date','end_date'])
        .str.strptime(pl.Date, '%m/%d/%Y'))
        .with_columns(unbilled_rec=calc_unbilled_pl())
    )

tweak_pl(bill_pl_lazy)

In [ ]:
tweak_pl(bill_pl_lazy).collect()

### Polars Questions

## DuckDB



DuckDB is an "in-process" SQL OLAP database management system.

* No depependencies (written in C++)
* No setup. No server (embedded)
* Support for complex queries, including window functions, CTEs, and subqueries
* Open Source (MIT License)


### Supporting Libraries

In [ ]:
!pip install -U jupysql

In [ ]:
%reload_ext sql
%config SqlMagic.autopandas=True
%config SqlMagic.feedback=False
%config SqlMagic.displaycon=True

In [ ]:
import duckdb

In [ ]:
%sql duckdb:///:memory: select 42

In [ ]:
%%sql duckdb:///:memory: 
select 42

In [ ]:
%%sql duckdb:///:memory: 
df << select 42

In [ ]:
df

### Load Data

In [ ]:
%%sql
DROP TABLE IF EXISTS bill;
CREATE TABLE bill AS FROM read_csv_auto('/tmp/bill_100k.csv')

In [ ]:
%%sql
SELECT * FROM bill LIMIT 5

In [ ]:
%%sql
FROM information_schema.tables



In [ ]:
%%sql
FROM information_schema.columns



### Complicated Query

In [ ]:
%%sql
SELECT 
    CASE
        WHEN (cancel_date IS NULL OR cancel_date >= period_start)
            AND start_date < period_start 
            AND end_date > period_start 
            AND rev > sum_payments THEN rev - sum_payments
        WHEN start_date < period_start AND end_date > period_start AND (cancel_date IS NULL OR cancel_date >= period_start) THEN 0
        ELSE NULL
    END as unbilled_rec
FROM bill;


### Arrow Integration

In [ ]:
import polars as pl

bill_pl = pl.read_csv('/tmp/bill_100k.csv')
bill_pl = tweak_pl(bill_pl)

In [ ]:
# DuckDB can read directly from Polars (or Pandas) dataframes
duckdb.sql('SELECT * FROM bill_pl LIMIT 5').pl()

In [ ]:
res = duckdb.sql('SELECT * FROM bill_pl LIMIT 5')

In [ ]:
# to pandas
res.df()

## Summary

![Desc.](chart.png)

## Questions


- *Effective Pandas 2* coming out soon
- *Effective Polars* in the works 

Please reach out if your team needs training with Pandas, Polars, or DuckDB.


matt@metasnake.com